Taking the top 500 albums of all time as ranked by Rolling Stone and finding the mean value after performing sentiment analysis on their lyrics

In [ ]:
# !pip install lyricsgenius
# !pip install vaderSentiment
# !pip install nltk
import nltk
import json
import requests
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from lyricsgenius import Genius
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np 
import requests
from io import BytesIO 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
from dotenv import load_dotenv
import string

In [ ]:
#load api key from .env file
load_dotenv()
genius = Genius('GENIUS_API_KEY')


#load the albumlist.csv into a dataframe
df = pd.read_csv('albumlist.csv')
df.head()

genius.remove_section_headers = True
genius.excluded_terms = ["(Remix)", "(Live)"]
genius.skip_non_songs = True

#remove the genre and subgenre columns from the dataframe
df = df.drop(['Genre', 'Subgenre'], axis=1)

#show the dataframe
df.head()

#create a new column called album_id and zero it out
df['album_id'] = 0

#show the dataframe
df.head()





In [ ]:
#load api key from .env file
load_dotenv()
genius = Genius('GENIUS_API_KEY')

#ask the user to input an artist and an album
artist = df['Artist'][0]
album = df['Album'][0]

#parameters for the api call
genius.remove_section_headers = True
genius.excluded_terms = ["(Remix)", "(Live)"]
genius.skip_non_songs = True

#search for the album


def search_album_id(album, artist):
    album = genius.search_albums(str(artist) + " " + str(album))
    album_names = []
    album_ids = []
    for i in range(len(album['sections'][0]['hits'])):
        album_names.append(album['sections'][0]['hits'][i]['result']['name'])
        album_ids.append(album['sections'][0]['hits'][i]['result']['id'])
    album_df = pd.DataFrame({'album_name': album_names, 'album_id': album_ids})
    album_df['choice'] = album_df.index
    album_df['choice'] = album_df['choice'] + 1
    return(album_df)





In [ ]:
search_album_id(album, artist)
pd.set_option('display.max_colwidth', 255)

#if album_name has any foreign language characters, remove that entry and reset the choice column
album_df = search_album_id(album, artist)
album_df['album_name'] = album_df['album_name'].str.replace(r'[^\x00-\x7F]+', '')
album_df = album_df[album_df['album_name'] != '']
album_df = album_df.reset_index(drop=True)
album_df['choice'] = album_df.index
album_df['choice'] = album_df['choice'] + 1

#if album name has any text matching 'Traduo', 'Traduccin', 'Portugus' or 'Espaol', remove that entry and reset the choice column
album_df = album_df[~album_df['album_name'].str.contains('Traduo')]
album_df = album_df[~album_df['album_name'].str.contains('Traduccin')]
album_df = album_df[~album_df['album_name'].str.contains('Portugus')]
album_df = album_df[~album_df['album_name'].str.contains('Espaol')]
album_df = album_df.reset_index(drop=True)
album_df['choice'] = album_df.index
album_df['choice'] = album_df['choice'] + 1

#show the dataframe
album_df



In [ ]:
#ask the user to choose an album based on the choices
#choice = int(input("Enter a choice: "))

#for the first 20 entries in the dataframe, search for the album id and append it to the dataframe under album_id. If the album id is not found or there is a 403 error, append 0 to the dataframe
from time import sleep


for i in range(499):
    artist = df['Artist'][i]
    album = df['Album'][i]
    try:
        album_df = search_album_id(album, artist)
        album_df['album_name'] = album_df['album_name'].str.replace(r'[^\x00-\x7F]+', '')
        album_df = album_df[album_df['album_name'] != '']
        album_df = album_df.reset_index(drop=True)
        album_df['choice'] = album_df.index
        album_df['choice'] = album_df['choice'] + 1
        album_df = album_df[~album_df['album_name'].str.contains('Traduo')]
        album_df = album_df[~album_df['album_name'].str.contains('Traduccin')]
        album_df = album_df[~album_df['album_name'].str.contains('Portugus')]
        album_df = album_df[~album_df['album_name'].str.contains('Espaol')]
        album_df = album_df.reset_index(drop=True)
        album_df['choice'] = album_df.index
        album_df['choice'] = album_df['choice'] + 1
        choice = 1
        df['album_id'][i] = album_df['album_id'][choice - 1]
        sleep(1)
    except:
        df['album_id'][i] = 0


#show the dataframe

df.head()










In [ ]:
#find every row where album_id is equal to 0 and add these to a new dataframe
df_zero = df[df['album_id'] == 0]

#show the dataframe
df_zero

def get_album(choice):
    album_name = df_zero.loc[df_zero['choice'] == choice, 'album_name'].iloc[0]
    album_id = df_zero.loc[df_zero['choice'] == choice, 'album_id'].iloc[0]
    return album_name, album_id

#search for the album
album = genius.search_albums(str(artist) + " " + str(album))

album_names = []
album_ids = []
for i in range(len(album['sections'][0]['hits'])):
    album_names.append(album['sections'][0]['hits'][i]['result']['name'])
    album_ids.append(album['sections'][0]['hits'][i]['result']['id'])
album_df = pd.DataFrame({'album_name': album_names, 'album_id': album_ids})
album_df['choice'] = album_df.index
album_df['choice'] = album_df['choice'] + 1

album_df
#ask the user to input a choice
choice = int(input("Enter a choice: "))
#call the get_album function
album_name, album_id = get_album(choice)
#display the album name and id``
print(album_name)
print(album_id)

In [ ]:
#export the dataframe to a csv file
df.to_csv('album_ids.csv', index=False)

#export the dataframe to a csv file
df_zero.to_csv('album_ids_zero.csv', index=False)



In [ ]:
def get_album(choice):
    album_name = album_df.loc[album_df['choice'] == choice, 'album_name'].iloc[0]
    album_id = album_df.loc[album_df['choice'] == choice, 'album_id'].iloc[0]
    return album_name, album_id

#ask the user to input a choice
choice = int(input("Enter a choice: "))
#call the get_album function
album_name, album_id = get_album(choice)




In [ ]:
#add the album_id to the initial dataframe at the correct location, as an integer
df.loc[0, 'album_id'] = int(album_id)

#show the dataframe
df.head()



In [ ]:
#for every row, collect the album_id based on user input from get_alubm function
for i in range(len(df)):
    artist = df.iloc[i]['Artist']
    album = df.iloc[i]['Album']
    album = genius.search_albums(str(artist) + " " + str(album))
    album_names = []
    album_ids = []
    for i in range(len(album['sections'][0]['hits'])):
        album_names.append(album['sections'][0]['hits'][i]['result']['name'])
        album_ids.append(album['sections'][0]['hits'][i]['result']['id'])
    album_df = pd.DataFrame({'album_name': album_names, 'album_id': album_ids})
    album_df['choice'] = album_df.index
    album_df['choice'] = album_df['choice'] + 1
    print(album_df)
    choice = int(input("Enter a choice: "))
    album_name, album_id = get_album(choice)
    print(album_name)
    print(album_id)
    df.loc[i, 'album_id'] = int(album_id)





In [ ]:
#import the album_ids.csv to a new dataframe and show it
df = pd.read_csv('album_ids_matched_manually.csv')
df.head()

#test first album_id row against the genius api to see if the album matches
album = genius.album(df['album_id'][0])
album

#fetch the full_title from the album_id row
album['album']['full_title']

#create a new column in the dataframe called full_title and set it to 0
df['full_title'] = 0


#show the dataframe
df.head()

#length of the dataframe
len(df)





In [ ]:
#create a column called "match" and set it to 0
#df['match'] = 0


#export the dataframe to a csv file to manually check the full_title column against the album column
#df.to_csv('album_ids_full_title.csv', index=False)



In [ ]:
# #import album_ids_matched_cleaned.csv to a new dataframe and show it
# df = pd.read_csv('album_ids_matched_cleaned.csv')
# df.head()

# #count all rows where match = 0
# df[df['match'] == 0].count()

# #create a new dataframe with all rows where match = 0
# df_zero = df[df['match'] == 0]

# #show the dataframe
# df_zero.head()

# #create a column called new_album_id and set it to 0 for all rows
# df_zero['new_album_id'] = 0

# #show the dataframe
# df_zero.head()


In [ ]:
#import album_ids_full_title.csv to a new dataframe called "master_df" and show it
master_df = pd.read_csv('album_ids_full_title.csv')
master_df.head()

#show all rows where match = 0
master_df[master_df['match'] == 0]

#export all rows where match = 0 to a csv file
master_df[master_df['match'] == 0].to_csv('album_ids_full_title_zero.csv', index=False)

#show all rows where match = 1
master_df[master_df['match'] == 1]

#export all rows where match = 1 to a csv file
master_df[master_df['match'] == 1].to_csv('album_ids_full_title_one.csv', index=False)

#remove all rows where match = 0 from the master_df dataframe
master_df = master_df[master_df['match'] == 1]

#show the master_df dataframe
master_df.head()

#count the rows in the master_df dataframe
len(master_df)


In [ ]:
master_df

#show master_df dataframe at iloc[0]
master_df.iloc[490]

In [ ]:
#album = genius.search_album(str(artist) + " " + str(album), album_id=album_id)
#album.save_lyrics('lyrics.json', overwrite=True)

import time

#iterate through the rows of the master_df dataframe and fetch the lyrics from the genius api using artist name, album name and album id. save the lyrics to a json file that is named based on the number of the loop. If there is a timeout, name the json file '404' and continue to the next loop
for i in range(len(master_df)):
    try:
        artist = master_df.iloc[i]['Artist']
        album = master_df.iloc[i]['Album']
        album_id = master_df.iloc[i]['album_id']
        album = genius.search_album(str(artist) + " " + str(album), album_id=album_id)
        album.save_lyrics(str(i) + '.json', overwrite=True)
        time.sleep(6)
    except:
        with open('404.json', 'w') as outfile:
            json.dump('404', outfile)
        time.sleep(6)
    


    





In [ ]:
master_df = pd.read_csv('album_ids_full_title.csv')
master_df.head()

In [ ]:
import glob
#count the number of json files in the directory
len(glob.glob1("","*.json"))


#create a new column in the dataframe called "has json" and set it to 0
master_df['has json'] = 0

#show the dataframe
master_df.head()

#count the number of json files in the directory and map the number column to the name of the json file, if there is a match, add it to the dataframe, else keep it as zero
for i in range(len(master_df)):
    if len(glob.glob(str(i) + '.json')) == 1:
        master_df['has json'][i] = 1
    else:
        master_df['has json'][i] = 0

#show the dataframe 
master_df.head()

#create a dataframe with all rows where has json = 0, keep all columns from master_df
master_df_zero = master_df[master_df['has json'] == 0]

#show the dataframe

master_df_zero.head()

#count the number of rows in the dataframe
len(master_df_zero)

#export the master_df_zero dataframe to a csv file
master_df_zero.to_csv('master_df_zero.csv', index=False)


#show the master_df dataframe

master_df.head()


In [ ]:
#import master_df_zero.csv to a new dataframe called "master_df_zero" and show it
master_df_zero = pd.read_csv('master_df_zero.csv')
master_df_zero.head()

master_df.tail(10)


In [ ]:
#for each album in the master_df_zero dataframe, search the genius api for the album name and artist name.
#If there is a match, fetch the album id and append it to the dataframe, else append a 404
for i in range(2):
    try:
        artist = master_df_zero.iloc[i]['Artist']
        album = master_df_zero.iloc[i]['Album']
        album_id = master_df_zero['album_id'][i]
        album = genius.search_album(album, artist, album_id=album_id)  #search for the album)
        #print the number in the dataframe, the album name, the artist name and the album id
    except:
        print('404')


In [ ]:
master_df_zero = pd.read_csv('master_df_zero.csv')
master_df_zero




In [ ]:
#set master_df to master_df_zero
master_df = master_df_zero

#show the dataframe
master_df

#for every number in the number column, subtract 1
for i in range(len(master_df)):
    master_df['Number'][i] = master_df['Number'][i] - 1

#show the dataframe
master_df

    



In [ ]:
import time

#iterate through the rows of the master_df dataframe and fetch the lyrics from the genius api using artist name, album name and album id. save the lyrics to a json file that is named based on the number of the loop. If there is a timeout, name the json file '404' and continue to the next loop
for i in range(len(master_df)):
    try:
        artist = master_df.iloc[i]['Artist']
        album = master_df.iloc[i]['Album']
        album_id = master_df.iloc[i]['album_id'].item()
        album = genius.search_album(str(album), str(artist), album_id=album_id)
        number = master_df.iloc[i]['Number']
        album.save_lyrics(str(number) + '.json', overwrite=True)
    except:
        with open('pee.json', 'w') as outfile:
            json.dump('pee', outfile)

         

In [ ]:
#impor album_ids_full_title.csv to a new dataframe called "use_this_df" and show it
use_this_df = pd.read_csv('album_ids_full_title.csv')
use_this_df.head()


#drop all rows where match = 0 from the use_this_df dataframe
use_this_df = use_this_df[use_this_df['match'] == 1]

#show the dataframe
use_this_df.head()

use_this_df




In [ ]:

artist = master_df.iloc[i]['Artist']
album = master_df.iloc[i]['Album']
album_id = master_df.iloc[i]['album_id'].item(0)
number = master_df.iloc[i]['Number']
#album_id = album_id.item()
#album = genius.search_album(str(album) + " " + str(artist), album_id=int(album_id))
       # album.save_lyrics(str(master_df.iloc[i]) + '.json', overwrite=True)

print(artist)
type(artist)
album_id
type(album_id)
type(number)

### Trying to add all the json files I have

## IMPORT AND START HERE


In [ ]:
#read the json file and print in a readable format and print keys
import json
with open('0.json') as f:
    data = json.load(f)
    # print(json.dumps(data, indent=4, sort_keys=True))
    # print(data.keys())

#make a new dataframe called "all" and add the columns "artist", "album", "album_id", "number", "song", "lyrics", "album_art", and "release_date"
all = pd.DataFrame(columns=['artist', 'album', 'album_id', 'number', 'song', 'lyrics', 'album_art'])


#read each json file in the directory and add the artist, album, album_id, number, song, lyrics, album_art, and release_date to the all dataframe
for i in range(len(glob.glob1("","*.json"))):
    try:
        with open(str(i) + '.json') as f:
            data = json.load(f)
            artist = data['artist']
            album = data['description_annotation']
            album_id = data['description_annotation']
            titles = []
            lyrics = []
            album_art = data['cover_art_url']
            number = i
        artist = artist['name']
        album = album['annotatable']['title']
        album_id = album_id['id']
    #add the output of the following to the titles list
        for i in range(len(data['tracks'])):
            print(data['tracks'][i]['song']['title'])
            titles.append(data['tracks'][i]['song']['title'])
        titles
        #fetch the lyrics for each song in the titles list and add it to the lyrics list
        for i in range(len(titles)):
            print(data['tracks'][i]['song']['lyrics'])
            lyrics.append(data['tracks'][i]['song']['lyrics'])
        lyrics

    #add the artist, album, album_id, titles, and lyrics to the all dataframe
        all = all.append({'artist': artist, 'album': album, 'album_id': album_id, 'number' : number, 'song': titles, 'lyrics': lyrics, 'album_art': album_art}, ignore_index=True)
    except:
        print('404')
#show the dataframe
all['lyrics']

In [ ]:
#write the all dataframe to a csv file called "all497.csv"
all.to_csv('MASTER_FINALVERSION.csv')

#check the dataframe album column for any duplicates and count them
all['album'].value_counts()

#find the album called "Sketches of Spain" and show the row
all['artist'].value_counts()

#find all artists with the string "Sly" in their name and show the rows
all[all['artist'].str.contains('Howl')]

#find the length of the all dataframe
len(all)




In [ ]:
all

In [ ]:
album_df = pd.read_csv('albumlist.csv')

album_df['Album'].value_counts()

#find rows where ALbum = Greatest Hits and show the dataframe
album_df[album_df['Artist'].str.contains('Howl')]

len(album_df)

This is a working cell to get the json schema

In [ ]:
import json
with open('0.json') as f:
    data = json.load(f)
    print(json.dumps(data, indent=4, sort_keys=True))
    print(data.keys())

In [ ]:
#create new dataframe from all that is named "all2" and includes artist,album, and album_id	
all2 = all[['artist', 'album', 'album_id']]

#show the dataframe
all2

#export all2 to a csv file called "all2.csv"
all2.to_csv('all2.csv')



In [ ]:
#count the json files in the directory from 1-100 and list the missing "gapped" files
for i in range(0,499):
    if os.path.isfile(str(i) + '.json'):
        pass
    else:
        print(i)



In [ ]:
#find if an album contains "layla" in the title
all[all['album'].str.contains('riot', case=False)]




In [ ]:
#initial album dataframe = albumlist.csv
album_df = pd.read_csv('albumlist.csv')

#show the dataframe
album_df


#find the iloc of the album containing the string "Proud Mary" and print it

album_df[album_df['Album'].str.contains('Proud Mary', case=False)].iloc[0]

#do the same for the album containing the string "Mr. Excitement"

album_df[album_df['Album'].str.contains('Mr. Excitement', case=False)].iloc[0]

#do the same for the album containing the string "Indestructible Beat"

album_df[album_df['Album'].str.contains('Indestructible Beat', case=False)].iloc[0]







In [ ]:
album_df.iloc[213]
album_df.iloc[235]
album_df.iloc[387]

#drop the rows at iloc 213, 235, and 387 from the album_df dataframe
album_df = album_df.drop([213, 235, 387])

#reset the index of the album_df dataframe
album_df = album_df.reset_index(drop=True)

#show the dataframe
album_df

#make every Number equal to the index of the dataframe plus 1   
album_df['Number'] = album_df.index + 1

#show the dataframe
album_df

#clean the text in the album and artist columns, make them all lowercase, and remove all punctuation
album_df['Album'] = album_df['Album'].str.lower()
album_df['Album'] = album_df['Album'].str.replace('[^\w\s]','')
album_df['Artist'] = album_df['Artist'].str.lower()
album_df['Artist'] = album_df['Artist'].str.replace('[^\w\s]','')

#show the dataframe
album_df


In [ ]:
#export album_df to a csv file calle "albumscleaned.csv"
album_df.to_csv('albumsremovecantfind.csv')


#clean the "all" dataframe the same way, make all text lowercase and remove all punctuation from the artist and album columns and make the Number column for the dataframe equal to the index plus 1
all['artist'] = all['artist'].str.lower()
all['artist'] = all['artist'].str.replace('[^\w\s]','')
all['album'] = all['album'].str.lower()
all['album'] = all['album'].str.replace('[^\w\s]','')
all['number'] = all.index + 1

#show the dataframe
all

#save the "all" dataframe to a csv file called "allinformationcleaned.csv"
all.to_csv('allinformationcleaned.csv')



In [ ]:
album_df

# Cleaning the lyrics

> This section is some regex and replacements to clean up the lyrics so we can prepare them to be tokenized for analysis

In [ ]:
len(all['lyrics'][0])
#print all the lyrics for the first entry in the dataframe

#create a new column in the all dataframe called "cleaned lyrics" and set it to 0
all['cleaned lyrics'] = 0

#each lyrics is a list, unlist it and set it to the cleaned lyrics column
all['cleaned lyrics'] = all['lyrics'].apply(lambda x: ' '.join(x))

#show the dataframe
all

#for each song in the song list in the dataframe, lets remove that string from the cleaned lyrics column
for i in range(len(all['song'])):
    for j in range(len(all['song'][i])):
        all['cleaned lyrics'][i] = all['cleaned lyrics'][i].replace(all['song'][i][j], '')
    
#continue cleaning the lyrics by removing the following strings
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('instrumental', '')
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('Lyrics', '')

#lets remove all newlines, tabs, and carriage returns
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\n', ' ')
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\t', ' ')
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\r', ' ')

all['cleaned lyrics'][0]

#there are particular strings in some lyrics that are stylize with "You might also like" followed by either a 1 digit or two digit number, lets remove those
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('You might also like\d', '')

#There are instances of "You might also likeEmbed" in the lyrics, lets remove those
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('You might also likeEmbed', '')

#remove all instances of a one digit number followed by "Embed" or a two digit number followed by "Embed" with no space in between
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\dEmbed', '')

#remove all instances of the word "Embed" followed by a one digit number or two digit number with no space in between
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('Embed\d', '')

#remove all instances of the word "Embed" connected to either a number or a letter with no space in between
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('Embed\w', '')

#remove any errant numbers that are connected to a letter, punctuation, bracket, *, or /
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\d[a-zA-Z\(\)\[\]\*\,\/]', '')

#remove the word "Embed" entirely
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('Embed', '')

#format the spacing in the lyrics to remove any extra spaces
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('  ', ' ')

#remove any extra spaces at the beginning of the lyrics
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('^\s+', '')

#remove any extra spaces at the end of the lyrics
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\s+$', '')

#remove all numbers in the lyrics
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('\d', '')

#remove all punctuation in the lyrics
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('[^\w\s]','')

#remove all instances of the word "lyrics" in the lyrics
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('lyrics', '')

#make sure there is only single spacing, remove all double spacing
all['cleaned lyrics'] = all['cleaned lyrics'].str.replace('  ', ' ')

all['cleaned lyrics'][0]

# Okay, we're pretty clean

> Let's prepare the lyrics for tokenization

In [ ]:
#create a new column in the all dataframe called "tokens" and set it to 0
all['tokens'] = 0

# Now let's make tokens from our cleaned lyrics

> We are going to be looking at the album as a whole for right now, so let's tokenize the entire lyrical space of the album and then find the sentiment score.

In [ ]:
#remove all punctuation, convert to lowercase, and remove stopwords, then tokenize the lyrics
all['tokens'] = all['cleaned lyrics'].apply(lambda x: word_tokenize(x.lower()))
all['tokens'] = all['tokens'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
all['tokens'] = all['tokens'].apply(lambda x: [word for word in x if word not in string.punctuation])

#show the dataframe
all
